In [2]:
# -*- coding: utf-8 -*-
"""step1_student_data.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1NG1zPbJTgl8GwuIAMicEZm9Fb07nCx5H
"""

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# *******************************************************************
# ** ส่วนที่เพิ่ม: ตั้งค่า Pandas เพื่อไม่ให้ตัดบรรทัดยาว ๆ **
# *******************************************************************
pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', 1000)
# *******************************************************************

# *******************************************************************
# ** ส่วนที่ 1: การโหลดและเตรียมข้อมูล student-mat.csv **
# *******************************************************************

# 1. โหลดไฟล์ CSV โดยใช้เซมิโคลอนเป็นตัวคั่น
df = pd.read_csv('student-mat.csv', sep=';')

# 2. ทำ One-Hot Encoding สำหรับตัวแปรที่เป็นข้อความ (Categorical)
# คอลัมน์ที่เป็นตัวเลขอยู่แล้ว (เช่น traveltime, studytime) จะถูกคงไว้
df_encoded = pd.get_dummies(df, drop_first=True)

# 3. กำหนด Target Column (ใช้ G3 เป็นคะแนนเกรดสุดท้าย)
TARGET_COLUMN = 'G3'

# X_df คือ Features (ทุกคอลัมน์ยกเว้นคอลัมน์เป้าหมาย)
X_df = df_encoded.drop(TARGET_COLUMN, axis=1)

# y_series คือ Target (แปลงคะแนนเกรดสุดท้าย G3 เป็น Binary: Pass/Fail
# สมมติว่าคะแนนผ่านคือ > 10
y = (df_encoded[TARGET_COLUMN] > 10).astype(int).values

# เตรียมตัวแปรให้พร้อมใช้งานกับโค้ดส่วนอื่น
X = X_df.values
feature_names = X_df.columns.tolist()

print("ข้อมูล Features ที่ถูกเข้ารหัส (ตัวอย่าง 5 แถวแรก)")
print(X_df.head())
print("\n")
# *******************************************************************


print(" MinMax (Manual Scaling) ")
# --- ส่วนที่ถูกแทนที่ด้วยการคำนวณ Manual ---
X_min = X.min(axis=0)
X_max = X.max(axis=0)

# คำนวณ Min-Max Scaling ด้วยตนเอง
denominator = X_max - X_min
# จัดการกรณีที่ X_max == X_min (ป้องกันหารด้วยศูนย์)
X_scaled = np.where(denominator == 0, 0, (X - X_min) / denominator)
# --- สิ้นสุดการคำนวณ Manual ---

# สร้าง DataFrame สำหรับข้อมูลที่ Scale แล้ว
df_scaled = pd.DataFrame(X_scaled, columns=feature_names)
print("ข้อมูล Features ที่ Scale แล้ว (ตัวอย่าง 5 แถวแรก)")
print(df_scaled.head())
print("\n")

print(" Variance ")
variances = X_scaled.var(axis=0)
# เนื่องจากมีคุณลักษณะเยอะมาก จึงแสดงแค่ 5 อันดับแรก
print("Top 5 Variances:")
for i in np.argsort(variances)[-5:]:
    print(feature_names[i], ":", variances[i])
print("\n")

# *******************************************************************
# ** ส่วนที่เพิ่มเข้าไปเพื่อแสดง Correlation Matrix เริ่มต้น **
# *******************************************************************
print("--- Correlation Matrix เริ่มต้น (ข้อมูลที่ถูกปรับมาตราส่วน) ---")
# แสดง Correlation Matrix ของคุณลักษณะทั้งหมด
correlation_matrix = df_scaled.corr()
print("ขนาดของ Matrix:", correlation_matrix.shape)
print(correlation_matrix)
print("\n")
# *******************************************************************

# *******************************************************************
# ** ส่วนที่เพิ่มเข้าไป: การลดคุณลักษณะตามค่าสหสัมพันธ์ (Correlation-Based Feature Selection) **
# *******************************************************************

def select_features_by_correlation(df, threshold=0.85):
    """
    ลดคุณลักษณะโดยกำจัดตัวที่มีสหสัมพันธ์สูงเกินเกณฑ์
    โดยเก็บตัวที่มีค่า Standard Deviation (S.D.) มากกว่าไว้
    """
    print("--- 🔴 เริ่มกระบวนการลดคุณลักษณะ (Threshold: > %.2f) 🔴 ---" % threshold)
    df_current = df.copy()

    round_count = 1

    while True:
        # 1. คำนวณ Correlation Matrix ของตัวแปรที่เหลืออยู่ทั้งหมด
        corr_matrix = df_current.corr().abs()

        # 2. ค้นหาค่าที่มากที่สุด (ที่ไม่ใช่แนวทแยงมุม)
        np.fill_diagonal(corr_matrix.values, 0)

        max_corr_value = corr_matrix.max().max()

        # 3. ตรวจสอบเงื่อนไข (Threshold)
        if max_corr_value <= threshold:
            print("\n✅ กระบวนการสิ้นสุด: ไม่มีคู่ใดที่มีสหสัมพันธ์เกินเกณฑ์ (", threshold, ")")

            # ** แสดงผล Correlation Matrix สุดท้าย **
            final_corr_matrix = df_current.corr()
            print("\n--- Correlation Matrix ของคุณลักษณะที่ถูกเลือก ---")
            print("ขนาดของ Matrix สุดท้าย:", final_corr_matrix.shape)
            print(final_corr_matrix)

            break

        # หาคู่ฟีเจอร์ที่มีสหสัมพันธ์สูงสุด
        r, c = np.unravel_index(corr_matrix.values.argmax(), corr_matrix.shape)
        feature_a = corr_matrix.columns[r]
        feature_b = corr_matrix.columns[c]

        print("\n--- รอบที่ %d ---" % round_count)

        # แสดงผล Correlation Matrix ของรอบปัจจุบัน (ขั้นตอนที่ 1)
        print("1. Correlation Matrix ปัจจุบัน (ค่าสัมบูรณ์สูงสุด %.4f):" % max_corr_value)
        # แสดงเฉพาะคู่ที่สัมพันธ์สูงสุด
        print(corr_matrix.loc[[feature_a, feature_b], [feature_a, feature_b]])

        # 4. เปรียบเทียบ S.D. (Standard Deviation)
        std_a = df_current[feature_a].std()
        std_b = df_current[feature_b].std()

        print("\n3. เปรียบเทียบ S.D. (ขั้นตอนการตัดสินใจ):")
        print("   * **%s**: S.D. = **%.5f**" % (feature_a, std_a))
        print("   * **%s**: S.D. = **%.5f**" % (feature_b, std_b))

        # การตัดสินใจ: เก็บตัวที่มี S.D. มากกว่า
        if std_a >= std_b:
            feature_to_drop = feature_b
            feature_to_keep = feature_a
        else:
            feature_to_drop = feature_a
            feature_to_keep = feature_b

        # ตัดตัวที่มีค่า S.D. น้อยกว่าทิ้ง
        df_current = df_current.drop(columns=[feature_to_drop])

        # 4. ผลการตัดสินใจ
        print("4. ผลการตัดสินใจ (ตัดสินใจตาม S.D.):")
        print("   * **เก็บ** ฟีเจอร์: '%s' (S.D. **%.5f**)" % (feature_to_keep, max(std_a, std_b)))
        print("   * **ตัดทิ้ง** ฟีเจอร์: '%s' (S.D. %.5f)" % (feature_to_drop, min(std_a, std_b)))

        round_count += 1

    print("--- 🏁 สิ้นสุดกระบวนการ 🏁 ---")
    print("คุณลักษณะที่เหลือ:", list(df_current.columns))
    return df_current

# เรียกใช้ฟังก์ชันเพื่อลดคุณลักษณะ
df_selected = select_features_by_correlation(df_scaled, threshold=0.90)
X_selected = df_selected.values
feature_names_selected = df_selected.columns.tolist()

print("\n--- ผลลัพธ์จากการลดคุณลักษณะ ---")
print("คุณลักษณะที่เลือก:", feature_names_selected)

# การทดสอบความแม่นยำด้วยคุณลักษณะที่เลือก
X_train, X_test, y_train, y_test_sel = train_test_split(X_selected, y, test_size=0.3, stratify=y)
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)
y_pred_sel = model.predict(X_test)
acc_sel = accuracy_score(y_test_sel, y_pred_sel)
print("Accuracy (Features Selected by Correlation/SD):", acc_sel)
print("\n")
# *******************************************************************


print(" Features ")
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, stratify=y)
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
acc_1 = accuracy_score(y_test, y_pred)
print("Accuracy (ใช้ทุก Features):", acc_1)
print("\n")

print("Threshold = 0.03")
selector = VarianceThreshold(threshold=0.03)
X_new_2 = selector.fit_transform(X_scaled)
X_train, X_test, y_train, y_test = train_test_split(X_new_2, y, test_size=0.3,  stratify=y)
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
acc_2 = accuracy_score(y_test, y_pred)
print("Accuracy:", acc_2)
print("\n")

print(" Best Features (อิงตาม Variance)")
# ต้องคำนวณ Variance ใหม่เนื่องจากมีคุณลักษณะที่เยอะขึ้น
sorted_idx = np.argsort(variances)
top3_idx = sorted_idx[-3:]
X_best3 = X_scaled[:, top3_idx]
X_train, X_test, y_train, y_test_3, = train_test_split(X_best3, y, test_size=0.3,  stratify=y)
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)
y_pred_3 = model.predict(X_test)
acc_3 = accuracy_score(y_test_3, y_pred_3)
print("Accuracy (Top 3):", acc_3)
print("\n")

# ... (ส่วนรายงานอื่น ๆ ยังคงอยู่) ...

ข้อมูล Features ที่ถูกเข้ารหัส (ตัวอย่าง 5 แถวแรก)
   age  Medu  Fedu  traveltime  studytime  failures  famrel  freetime  goout  Dalc  Walc  health  absences  G1  G2  school_MS  sex_M  address_U  famsize_LE3  Pstatus_T  Mjob_health  Mjob_other  Mjob_services  Mjob_teacher  Fjob_health  Fjob_other  Fjob_services  Fjob_teacher  reason_home  reason_other  reason_reputation  guardian_mother  guardian_other  schoolsup_yes  famsup_yes  paid_yes  activities_yes  nursery_yes  higher_yes  internet_yes  romantic_yes
0   18     4     4           2          2         0       4         3      4     1     1       3         6   5   6      False  False       True        False      False        False       False          False         False        False       False          False          True        False         False              False             True           False           True       False     False           False         True        True         False         False
1   17     1     1         

In [8]:
# -*- coding: utf-8 -*-
"""step1_student_data.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1NG1zPbJTgl8GwuIAMicEZm9Fb07nCx5H
"""

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# *******************************************************************
# ** ส่วนที่เพิ่ม: ตั้งค่า Pandas เพื่อไม่ให้ตัดบรรทัดยาว ๆ **
# *******************************************************************
pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)
# *******************************************************************

# *******************************************************************
# ** ส่วนที่ 1: การโหลดและเตรียมข้อมูล student-mat.csv **
# *******************************************************************

# 1. โหลดไฟล์ CSV โดยใช้เซมิโคลอนเป็นตัวคั่น
df = pd.read_csv('student-mat.csv', sep=';')

# 2. ทำ One-Hot Encoding สำหรับตัวแปรที่เป็นข้อความ (Categorical)
df_encoded = pd.get_dummies(df, drop_first=True)

# 3. กำหนด Target Column (ใช้ G3 เป็นคะแนนเกรดสุดท้าย)
TARGET_COLUMN = 'G3'

# X_df คือ Features (ทุกคอลัมน์ยกเว้นคอลัมน์เป้าหมาย)
X_df = df_encoded.drop(TARGET_COLUMN, axis=1)

# y_series คือ Target (แปลงคะแนนเกรดสุดท้าย G3 เป็น Binary: Pass/Fail
# สมมติว่าคะแนนผ่านคือ > 10
y = (df_encoded[TARGET_COLUMN] > 10).astype(int).values

# เตรียมตัวแปรให้พร้อมใช้งานกับโค้ดส่วนอื่น
X = X_df.values
feature_names = X_df.columns.tolist()

print("ข้อมูล Features ที่ถูกเข้ารหัส (ตัวอย่าง 5 แถวแรก)")
print(X_df.head())
print("\n")
# *******************************************************************


print(" MinMax (Manual Scaling) ")
# --- ส่วนที่ถูกแทนที่ด้วยการคำนวณ Manual ---
X_min = X.min(axis=0)
X_max = X.max(axis=0)

# คำนวณ Min-Max Scaling ด้วยตนเอง
denominator = X_max - X_min
# จัดการกรณีที่ X_max == X_min (ป้องกันหารด้วยศูนย์)
X_scaled = np.where(denominator == 0, 0, (X - X_min) / denominator)
# --- สิ้นสุดการคำนวณ Manual ---

# สร้าง DataFrame สำหรับข้อมูลที่ Scale แล้ว
df_scaled = pd.DataFrame(X_scaled, columns=feature_names)
print("ข้อมูล Features ที่ Scale แล้ว (ตัวอย่าง 5 แถวแรก)")
print(df_scaled.head())
print("\n")

print(" Variance ")
variances = X_scaled.var(axis=0)
print("Top 5 Variances:")
for i in np.argsort(variances)[-5:]:
    print(feature_names[i], ":", variances[i])
print("\n")

# *******************************************************************
# ** ส่วนที่เพิ่มเข้าไปเพื่อแสดง Correlation Matrix เริ่มต้น **
# *******************************************************************
print("--- Correlation Matrix เริ่มต้น (ข้อมูลที่ถูกปรับมาตราส่วน) ---")
correlation_matrix = df_scaled.corr()
print("ขนาดของ Matrix:", correlation_matrix.shape)
print(correlation_matrix)
print("\n")
# *******************************************************************

# *******************************************************************
# ** ส่วนที่เพิ่มเข้าไป: การลดคุณลักษณะตามค่าสหสัมพันธ์ (Correlation-Based Feature Selection) **
# *******************************************************************

def select_features_by_correlation(df, threshold=0.65):
    """
    ลดคุณลักษณะโดยกำจัดตัวที่มีสหสัมพันธ์สูงเกินเกณฑ์
    โดยเก็บตัวที่มีค่า Standard Deviation (S.D.) มากกว่าไว้
    """
    print("--- 🔴 เริ่มกระบวนการลดคุณลักษณะ (Threshold: > %.2f) 🔴 ---" % threshold)
    df_current = df.copy()

    round_count = 1

    while True:
        # 1. คำนวณ Correlation Matrix ของตัวแปรที่เหลืออยู่ทั้งหมด
        corr_matrix = df_current.corr().abs()

        # 2. ค้นหาค่าที่มากที่สุด (ที่ไม่ใช่แนวทแยงมุม)
        np.fill_diagonal(corr_matrix.values, 0)

        max_corr_value = corr_matrix.max().max()

        # 3. ตรวจสอบเงื่อนไข (Threshold)
        if max_corr_value <= threshold:
            print("\n✅ กระบวนการสิ้นสุด: ไม่มีคู่ใดที่มีสหสัมพันธ์เกินเกณฑ์ (", threshold, ")")

            # ** แสดงผล Correlation Matrix สุดท้าย **
            final_corr_matrix = df_current.corr()
            print("\n--- Correlation Matrix ของคุณลักษณะที่ถูกเลือก ---")
            print("ขนาดของ Matrix สุดท้าย:", final_corr_matrix.shape)
            print(final_corr_matrix)

            break

        # หาคู่ฟีเจอร์ที่มีสหสัมพันธ์สูงสุด
        r, c = np.unravel_index(corr_matrix.values.argmax(), corr_matrix.shape)
        feature_a = corr_matrix.columns[r]
        feature_b = corr_matrix.columns[c]

        print("\n--- รอบที่ %d ---" % round_count)

        # แสดงผล Correlation Matrix ของรอบปัจจุบัน (ขั้นตอนที่ 1)
        print("1. Correlation Matrix ปัจจุบัน (ค่าสัมบูรณ์สูงสุด %.4f):" % max_corr_value)
        # แสดงเฉพาะคู่ที่สัมพันธ์สูงสุด
        print(corr_matrix.loc[[feature_a, feature_b], [feature_a, feature_b]])

        # 4. เปรียบเทียบ S.D. (Standard Deviation)
        std_a = df_current[feature_a].std()
        std_b = df_current[feature_b].std()

        print("\n3. เปรียบเทียบ S.D. (ขั้นตอนการตัดสินใจ):")
        print("   * **%s**: S.D. = **%.5f**" % (feature_a, std_a))
        print("   * **%s**: S.D. = **%.5f**" % (feature_b, std_b))

        # การตัดสินใจ: เก็บตัวที่มี S.D. มากกว่า
        if std_a >= std_b:
            feature_to_drop = feature_b
            feature_to_keep = feature_a
        else:
            feature_to_drop = feature_a
            feature_to_keep = feature_b

        # ตัดตัวที่มีค่า S.D. น้อยกว่าทิ้ง
        df_current = df_current.drop(columns=[feature_to_drop])

        # 4. ผลการตัดสินใจ
        print("4. ผลการตัดสินใจ (ตัดสินใจตาม S.D.):")
        print("   * **เก็บ** ฟีเจอร์: '%s' (S.D. **%.5f**)" % (feature_to_keep, max(std_a, std_b)))
        print("   * **ตัดทิ้ง** ฟีเจอร์: '%s' (S.D. %.5f)" % (feature_to_drop, min(std_a, std_b)))

        round_count += 1

    print("--- 🏁 สิ้นสุดกระบวนการ 🏁 ---")

    return df_current

# เรียกใช้ฟังก์ชันเพื่อลดคุณลักษณะ
df_selected = select_features_by_correlation(df_scaled, threshold=0.45)
X_selected = df_selected.values
feature_names_selected = df_selected.columns.tolist()

# ----------------------------------------------------
# ** ส่วนแสดงผลลัพธ์สุดท้ายที่จัดเรียงตาม S.D. **
# ----------------------------------------------------
# แก้ไขข้อผิดพลาด: ใช้ df_selected.std() แทน np.sqrt(df_selected.var())
final_stds = df_selected.std(axis=0)

result_summary = pd.DataFrame({
    'Feature': feature_names_selected,
    'Standard Deviation (S.D.)': final_stds.values
}).sort_values(by='Standard Deviation (S.D.)', ascending=False).reset_index(drop=True)


print("\n--- 🏆 ผลลัพธ์จากการลดคุณลักษณะ (จัดเรียงตาม S.D.) 🏆 ---")
print("คุณลักษณะที่เหลือทั้งหมด: %d ตัว" % len(feature_names_selected))
print(result_summary)
print("\n")
# ----------------------------------------------------

# การทดสอบความแม่นยำด้วยคุณลักษณะที่เลือก
X_train, X_test, y_train, y_test_sel = train_test_split(X_selected, y, test_size=0.3, stratify=y)
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)
y_pred_sel = model.predict(X_test)
acc_sel = accuracy_score(y_test_sel, y_pred_sel)
print("Accuracy (Features Selected by Correlation/SD):", acc_sel)
print("\n")
# *******************************************************************


print(" Features ")
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, stratify=y)
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
acc_1 = accuracy_score(y_test, y_pred)
print("Accuracy (ใช้ทุก Features):", acc_1)
print("\n")

print("Threshold = 0.03")
selector = VarianceThreshold(threshold=0.03)
X_new_2 = selector.fit_transform(X_scaled)
X_train, X_test, y_train, y_test = train_test_split(X_new_2, y, test_size=0.3,  stratify=y)
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
acc_2 = accuracy_score(y_test, y_pred)
print("Accuracy:", acc_2)
print("\n")

print(" Best Features (อิงตาม Variance)")
# ใช้ Variances ที่คำนวณไว้ก่อนหน้า
sorted_idx = np.argsort(variances)
top3_idx = sorted_idx[-3:]
X_best3 = X_scaled[:, top3_idx]
X_train, X_test, y_train, y_test_3, = train_test_split(X_best3, y, test_size=0.3,  stratify=y)
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)
y_pred_3 = model.predict(X_test)
acc_3 = accuracy_score(y_test_3, y_pred_3)
print("Accuracy (Top 3):", acc_3)
print("\n")

ข้อมูล Features ที่ถูกเข้ารหัส (ตัวอย่าง 5 แถวแรก)
   age  Medu  Fedu  traveltime  studytime  failures  famrel  freetime  goout  Dalc  Walc  health  absences  G1  G2  school_MS  sex_M  address_U  famsize_LE3  Pstatus_T  Mjob_health  Mjob_other  Mjob_services  Mjob_teacher  Fjob_health  Fjob_other  Fjob_services  Fjob_teacher  reason_home  reason_other  reason_reputation  guardian_mother  guardian_other  schoolsup_yes  famsup_yes  paid_yes  activities_yes  nursery_yes  higher_yes  internet_yes  romantic_yes
0   18     4     4           2          2         0       4         3      4     1     1       3         6   5   6      False  False       True        False      False        False       False          False         False        False       False          False          True        False         False              False             True           False           True       False     False           False         True        True         False         False
1   17     1     1         